# Capstone Project - Predicting Suitable location for a New Chinese Restaurant in Queens, New York, USA
### Applied Data Science Capstone Project

## Table of contents
* [Introduction: Problem Statement](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)



## Introduction: Problem Statement <a name="introduction"></a>

Good health is very important to human life and food contribute immensely to this. Ingestion of good/healthy food is very important for sustainability of life and good health. There are quite a number of food outlets at good locations in New York. In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a Chinese restaurant in New York City, United States of America

Since there are lots of restaurants in New York City, we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Chinese restaurants in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decision are:

* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Chinese restaurants in the neighborhood, if any
* distance of neighborhood from city center
* We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Python library geopy/**Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API
* coordinate of New York center will be obtained using **Google Maps API geocoding** of Queens




### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around New York city center.

Let's first find the latitude & longitude of New York city center, using specific, well known address and Google Maps geocoding API.

In [4]:
#Hidden Cell

In [2]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Queens, New York, USA'
ny_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, ny_center))

Coordinate of Queens, New York, USA: [40.7282239, -73.7948516]


In the code below, we will create grid of area candidates, which will be spaced equally and  within 4km from Queens.
Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [ ]:
#!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print(' center longitude={}, latitude={}'.format(ny_center[1], ny_center[0]))
x, y = lonlat_to_xy(ny_center[1], ny_center[0])
print('New York center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('New York center longitude={}, latitude={}'.format(lo, la))

Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all its neighbors**.

In [5]:
ny_center_x, ny_center_y = lonlat_to_xy(ny_center[1], ny_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = ny_center_x - 4000
x_step = 600
y_min = ny_center_y - 4000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(ny_center_x, ny_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

301 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [6]:
#!pip install folium

import folium

In [7]:
map_ny = folium.Map(location=ny_center, zoom_start=13)
folium.Marker(ny_center, popup='Queens').add_to(map_ny)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_ny) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_ny)
    #folium.Marker([lat, lon]).add_to(map_ny)
map_ny

Now we have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~4km from Queens. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [8]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, ny_center[0], ny_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(ny_center[0], ny_center[1], addr))

Reverse geocoding check
-----------------------
Address of [40.7282239, -73.7948516] is: LaGuardia, Flushing, NY 11366, USA


In [9]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', USA', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [10]:
addresses[1:20]

['100-14 196th St, Jamaica, NY 11423',
 '196-31 Hiawatha Ave, Hollis, NY 11423',
 '197-25 90th Ave, Jamaica, NY 11423',
 'Hillside Av/202 St, Queens, NY 11423',
 'Francis Lewis Blvd/204 St, Queens, NY 11427',
 '8030 208th St, Queens Village, NY 11427',
 '83-05 Union Tpke, Fresh Meadows, NY 11366',
 '79-31 209th St, Flushing, NY 11364',
 '210-10 75th Ave, Oakland Gardens, NY 11364',
 '69-10 213th St, Oakland Gardens, NY 11364',
 '64-40 215th St, Flushing, NY 11364',
 '216-9 Horace Harding Expy, Bayside, NY 11364',
 '56-07 218th St, Flushing, NY 11364',
 '105-48 Farmers Blvd, Jamaica, NY 11412',
 '190-17 103rd Ave, Jamaica, NY 11423',
 '91-60 193rd St, Hollis, NY 11423',
 '195-05 90th Ave, Jamaica, NY 11423',
 '196-21 Hillside Avenue, Jamaica, NY 11423',
 '198-25 Pompeii Ave, Jamaica, NY 11423']

Let's place all into a Pandas dataframe.

In [11]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"194-02 109th Ave, Jamaica, NY 11412",40.705618,-73.762994,-5.822565e+06,9.838198e+06,5623.919307
1,"100-14 196th St, Jamaica, NY 11423",40.709152,-73.762841,-5.821965e+06,9.838198e+06,5248.663484
2,"196-31 Hiawatha Ave, Hollis, NY 11423",40.712686,-73.762688,-5.821365e+06,9.838198e+06,4918.177342
3,"197-25 90th Ave, Jamaica, NY 11423",40.716220,-73.762535,-5.820765e+06,9.838198e+06,4642.032784
4,"Hillside Av/202 St, Queens, NY 11423",40.719754,-73.762382,-5.820165e+06,9.838198e+06,4430.402732
5,"Francis Lewis Blvd/204 St, Queens, NY 11427",40.723289,-73.762229,-5.819565e+06,9.838198e+06,4292.839197
6,"8030 208th St, Queens Village, NY 11427",40.726823,-73.762076,-5.818965e+06,9.838198e+06,4236.563274
7,"83-05 Union Tpke, Fresh Meadows, NY 11366",40.730358,-73.761923,-5.818365e+06,9.838198e+06,4264.794060
8,"79-31 209th St, Flushing, NY 11364",40.733894,-73.761769,-5.817765e+06,9.838198e+06,4375.896293
9,"210-10 75th Ave, Oakland Gardens, NY 11364",40.737429,-73.761616,-5.817165e+06,9.838198e+06,4563.821685


...and let's now save/persist this data into local file.

In [12]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants. We will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Chinese restaurant' category, as we need info on Chinese restaurants in the neighborhood.

Foursquare credentials are defined in hidden cell bellow.

In [5]:
# Hidden Cell


Category IDs corresponding to Chinese restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

In [18]:

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

chinese_restaurant_categories = ['4bf58dd8d48988d110941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [19]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    chinese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, client_id, client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_chinese = is_restaurant(venue_categories, specific_filter=chinese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_chinese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_chinese:
                    chinese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, chinese_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
chinese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('chinese_restaurants_350.pkl', 'rb') as f:
        chinese_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, chinese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('chinese_restaurants_350.pkl', 'wb') as f:
        pickle.dump(chinese_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [20]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Chinese restaurants:', len(chinese_restaurants))
print('Percentage of Chinese restaurants: {:.2f}%'.format(len(chinese_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 485
Total number of Chinese restaurants: 17
Percentage of Chinese restaurants: 3.51%
Average number of restaurants in neighborhood: 2.5813953488372094


In [21]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('51eebc97498e71e3dcb8eeca', "Brook & AJ's", 40.70529843503885, -73.76290003509328, '190 07 109th Rd, Saint Albans, NY 11412, United States', 36, False, -5822618.831038998, 9838184.103877984)
('4e4dfb24bd4101d0d79f8cf5', 'New Ho Wah', 40.71234422072162, -73.7644111194512, '19519 Jamaica Ave, Hollis, NY 11423, United States', 309, False, -5821430.209004179, 9838418.478027312)
('4c1253b7a5eb76b065ecbdb7', "Cindee's Bakery", 40.712366154300774, -73.76449484024158, '198-06 Jamaica ave, Queens, NY, United States', 303, False, -5821426.844508159, 9838429.404814983)
('572539e2498e0bb2afa6c333', 'Safari Carribean American Restaurant', 40.713524, -73.76308, '196 23 Jamaica Avenue (196ST), Jamaica, NY 11423, United States', 303, False, -5821224.467680871, 9838253.282527644)
('4f580c1be4b0bdfd0e7e8102', 'Rajdhani Indian Restaurant', 40.72114845086291, -73.75923501183439, '206-12 Hillside Ave, Queens Village, NY 11427, United States', 347, False, -58

In [22]:
print('List of Chinese restaurants')
print('---------------------------')
for r in list(chinese_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(chinese_restaurants))

List of Chinese restaurants
---------------------------
('4b9c519ef964a520065f36e3', 'Villa Rustica Ristorante & Pizza', 40.74015832916982, -73.75805263301491, '73-28 Bell Blvd, Bayside, NY 11364, United States', 301, True, -5816687.231366556, 9837753.795940802)
('4c1e3a9f8b3aa593b4b49b5f', "L'italiana Trattoria", 40.74787714066193, -73.7614811585113, '21601 Horace Harding Expy, Bayside Hills, NY 11364, United States', 32, True, -5815393.540334843, 9838238.93873659)
('4f324cd519836c91c7ca1e57', "Anthony's", 40.732124, -73.76844, '22202 Union Tpke, Oakland Gardens, NY 11364, United States', 248, True, -5818093.287426579, 9839048.37444471)
('54c3d875498efb477dbc5647', 'House DeSimone', 40.749353, -73.766524, 'Queens, NY, United States', 344, True, -5815164.789339939, 9838897.13330277)
('4c2df17d8a349521f649cc9f', 'Nino"s Pizzeria', 40.712739341, -73.78357433, '17919 Hillside Ave, Jamaica, NY 11432, United States', 256, True, -5821443.922458522, 9840893.750757989)
('4b368ed8f964a520f53725

In [23]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 
Restaurants around location 102: Peking Restaurant Fresh Mdws, Peking, Peking Chinese Restaurant
Restaurants around location 103: QDOBA Mexican Eats, Boston Market, Applebee's Grill + Bar, Eons Greek Food For Life, Pokework
Restaurants around location 104: QDOBA Mexican Eats, Boston Market, Hooters, Applebee's Grill + Bar, Pokework, Eons Greek Food For Life, Dai Wong
Restaurants around location 105: 
Restaurants around location 106: 
Restaurants around location 107: Piccolo Sogno, Fortune Garden
Restaurants around location 108: Mythos, Fontana Famous Pizza & Gyro, North Shore Diner, Gyro World, TANG, Chicken Festival, Darouso's
Restaurants around location 109: Mythos, Gyro World, Fontana Famous Pizza & Gyro, North Shore Diner, TANG, Galbi Ma Eul, El Patron Mexican Grill, Chicken Festival
Restaurants around location 110: Tierras Centro Americanas, joey pizza Jamaica ave 166-14


Let's now see all the collected restaurants in our area of interest on map, and let's also show Chinese restaurants in different color.

In [24]:
map_queens = folium.Map(location=ny_center, zoom_start=13)
folium.Marker(ny_center, popup='Queens').add_to(map_queens)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_chinese = res[6]
    color = 'red' if is_chinese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_queens)
map_queens

We have all the restaurants in area within few kilometers from Queens, and we know which ones are Chinese restaurants. We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Chinese restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Queens, New York that have low restaurant density, particularly those with low number of Chinese restaurants. We will limit our analysis to area ~4km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 4km from Queens, New York center**. We have also **identified Chinese restaurants** 

In the second step of the analysis,we will calculate and explore '**restaurant density**' across different areas of Queens - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Chinese restaurants in vicinity) and focus our attention on those areas.

In the third step we will focus on most promising areas create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **no more than two restaurants in radius of 250 meters**, and we want locations **without Chinese restaurants in radius of minimum of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [25]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 2.5813953488372094


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"194-02 109th Ave, Jamaica, NY 11412",40.705618,-73.762994,-5.822565e+06,9.838198e+06,5623.919307,1
1,"100-14 196th St, Jamaica, NY 11423",40.709152,-73.762841,-5.821965e+06,9.838198e+06,5248.663484,0
2,"196-31 Hiawatha Ave, Hollis, NY 11423",40.712686,-73.762688,-5.821365e+06,9.838198e+06,4918.177342,2
3,"197-25 90th Ave, Jamaica, NY 11423",40.716220,-73.762535,-5.820765e+06,9.838198e+06,4642.032784,0
4,"Hillside Av/202 St, Queens, NY 11423",40.719754,-73.762382,-5.820165e+06,9.838198e+06,4430.402732,3
5,"Francis Lewis Blvd/204 St, Queens, NY 11427",40.723289,-73.762229,-5.819565e+06,9.838198e+06,4292.839197,0
6,"8030 208th St, Queens Village, NY 11427",40.726823,-73.762076,-5.818965e+06,9.838198e+06,4236.563274,0
7,"83-05 Union Tpke, Fresh Meadows, NY 11366",40.730358,-73.761923,-5.818365e+06,9.838198e+06,4264.794060,0
8,"79-31 209th St, Flushing, NY 11364",40.733894,-73.761769,-5.817765e+06,9.838198e+06,4375.896293,1
9,"210-10 75th Ave, Oakland Gardens, NY 11364",40.737429,-73.761616,-5.817165e+06,9.838198e+06,4563.821685,0


OK, now let's calculate the **distance to nearest Chinese restaurant from every area candidate center** 

In [26]:
distances_to_chinese_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in chinese_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_chinese_restaurant.append(min_distance)

df_locations['Distance to Chinese restaurant'] = distances_to_chinese_restaurant

In [27]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Chinese restaurant
0,"194-02 109th Ave, Jamaica, NY 11412",40.705618,-73.762994,-5.822565e+06,9.838198e+06,5623.919307,1,2919.560411
1,"100-14 196th St, Jamaica, NY 11423",40.709152,-73.762841,-5.821965e+06,9.838198e+06,5248.663484,0,2745.643694
2,"196-31 Hiawatha Ave, Hollis, NY 11423",40.712686,-73.762688,-5.821365e+06,9.838198e+06,4918.177342,2,2696.902964
3,"197-25 90th Ave, Jamaica, NY 11423",40.716220,-73.762535,-5.820765e+06,9.838198e+06,4642.032784,0,2779.930197
4,"Hillside Av/202 St, Queens, NY 11423",40.719754,-73.762382,-5.820165e+06,9.838198e+06,4430.402732,3,2239.432346
5,"Francis Lewis Blvd/204 St, Queens, NY 11427",40.723289,-73.762229,-5.819565e+06,9.838198e+06,4292.839197,0,1699.712180
6,"8030 208th St, Queens Village, NY 11427",40.726823,-73.762076,-5.818965e+06,9.838198e+06,4236.563274,0,1217.779028
7,"83-05 Union Tpke, Fresh Meadows, NY 11366",40.730358,-73.761923,-5.818365e+06,9.838198e+06,4264.794060,0,892.720575
8,"79-31 209th St, Flushing, NY 11364",40.733894,-73.761769,-5.817765e+06,9.838198e+06,4375.896293,1,911.545001
9,"210-10 75th Ave, Oakland Gardens, NY 11364",40.737429,-73.761616,-5.817165e+06,9.838198e+06,4563.821685,0,652.355459


In [28]:
print('Average distance to closest Chinese restaurant from each area center:', df_locations['Distance to Chinese restaurant'].mean())

Average distance to closest Chinese restaurant from each area center: 1385.9267244090722


OK, so **on average Chinese restaurant can be found within ~1386m** from every area center candidate.

Let's crete a map showing **heatmap / density of restaurants** and try to extract some meaningfull info from that. 

In [29]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

chinese_latlons = [[res[2], res[3]] for res in chinese_restaurants.values()]

In [30]:
from folium import plugins
from folium.plugins import HeatMap

map_queens = folium.Map(location=ny_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_queens) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_queens)
folium.Marker(ny_center).add_to(map_queens)
folium.Circle(ny_center, radius=1000, fill=False, color='white').add_to(map_queens)
folium.Circle(ny_center, radius=2000, fill=False, color='white').add_to(map_queens)
folium.Circle(ny_center, radius=3000, fill=False, color='white').add_to(map_queens)

map_queens


Let's create another heatmap map showing **heatmap/density of Chinese restaurants** only.

In [31]:
map_queens = folium.Map(location=ny_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_queens) #cartodbpositron cartodbdark_matter
HeatMap(chinese_latlons).add_to(map_queens)
folium.Marker(ny_center).add_to(map_queens)
folium.Circle(ny_center, radius=1000, fill=False, color='white').add_to(map_queens)
folium.Circle(ny_center, radius=2000, fill=False, color='white').add_to(map_queens)
folium.Circle(ny_center, radius=3000, fill=False, color='white').add_to(map_queens)

map_queens

Let us detect the number of restaurants that are nearby with minimum distance of 3km

In [32]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 3000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

restaurant_counts = []
chinese_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(xs, ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, chinese_restaurants)
    chinese_distances.append(distance)
print('done.')

Generating data on location candidates... done.


In [33]:
# Let's put this into dataframe
df_new_locations = pd.DataFrame({'Latitude':latitudes,
                                 'Longitude':longitudes,
                                 'X':xs,
                                 'Y':ys,
                                 'Restaurants nearby':restaurant_counts,
                                 'Distance to Chinese restaurant':chinese_distances})

df_new_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Chinese restaurant
0,40.705618,-73.762994,-5.822565e+06,9.838198e+06,1,2919.560411
1,40.709152,-73.762841,-5.821965e+06,9.838198e+06,0,2745.643694
2,40.712686,-73.762688,-5.821365e+06,9.838198e+06,3,2696.902964
3,40.716220,-73.762535,-5.820765e+06,9.838198e+06,0,2779.930197
4,40.719754,-73.762382,-5.820165e+06,9.838198e+06,0,2239.432346
5,40.723289,-73.762229,-5.819565e+06,9.838198e+06,0,1699.712180
6,40.726823,-73.762076,-5.818965e+06,9.838198e+06,0,1217.779028
7,40.730358,-73.761923,-5.818365e+06,9.838198e+06,0,892.720575
8,40.733894,-73.761769,-5.817765e+06,9.838198e+06,0,911.545001
9,40.737429,-73.761616,-5.817165e+06,9.838198e+06,0,652.355459


Now we are going to **filter** those locations: we're interested only in **locations with no more than two restaurants in radius of 250 meters**, and **no Chinese restaurants in radius of 400 meters**.

In [34]:
good_res_count = np.array((df_new_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_new_locations['Distance to Chinese restaurant']>=400)
print('Locations with no Chinese restaurants within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_new_locations[good_locations]


Locations with no more than two restaurants nearby: 269
Locations with no Chinese restaurants within 400m: 281
Locations with both conditions met: 255


Let's see how this looks on a map.

In [35]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_queens = folium.Map(location=ny_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_queens)
HeatMap(restaurant_latlons).add_to(map_queens)
folium.Circle(ny_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_queens)
folium.Marker(ny_center).add_to(map_queens)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_queens) 

map_queens

Now we have a bunch of locations fairly close to Queens, and we know that each of those locations has no more than two restaurants in radius of 250m, and no Chinese restaurant closer than 400m. Any of those locations is a potential candidate for a new Chinese restaurant, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [36]:
map_queens = folium.Map(location=ny_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_queens)
folium.Marker(ny_center).add_to(map_queens)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_queens)

map_queens

Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* Chinese restaurants at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [37]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_queens = folium.Map(location=ny_center, zoom_start=14)
folium.Marker(ny_center).add_to(map_queens)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_queens)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_queens)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_queens) 

map_queens

Our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

A good place to start exploring the neighborhood is the addresses of these clusters so as to know which is best suited for a new Chinese Restaurant



Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [38]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', USA', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, ny_center_x, ny_center_y)
    print('{}{} => {:.1f}km from Queens'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

154-44 71st Ave, Flushing, NY 11367                => 2.5km from Queens
86-31 Sancho St, Jamaica, NY 11423                 => 3.3km from Queens
Grand Central Pkwy, Jamaica, NY 11435              => 4.5km from Queens
46-61 189th St, Flushing, NY 11358                 => 3.9km from Queens
172-28 83rd Ave, Jamaica, NY 11432                 => 1.8km from Queens
18402 Jamaica Ave, Queens, NY 11423                => 4.2km from Queens
67-1 210th St, Oakland Gardens, NY 11364           => 4.4km from Queens
164 St/Booth Memorial Av, Queens, NY 11355         => 2.8km from Queens
14453 85th Ave, Briarwood, NY 11435                => 3.5km from Queens
45-28 160th St, Flushing, NY 11358                 => 4.9km from Queens
174-20 Jewel Ave, Flushing, NY 11365               => 0.5km from Queens
140-15 58th Rd, Flushing, NY 11355                 => 4.8km from Queens
Unnamed Road, Flushing, NY 11367                   => 5.0km from Queens


This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants and no Chinese restaurants nearby, all zones being fairly close to city center. The centers/addresses of these zones in green circles on the map should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations.

In [40]:
map_queens = folium.Map(location=ny_center, zoom_start=14)
folium.Circle(ny_center, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_queens)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_queens) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_queens)
map_queens

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in New York, there are pockets of low restaurant density fairly close to the city centre. 

After directing our attention to areas with low restaurant in Queens, we first created a dense grid of location candidates; those locations were then filtered so that those with more than two restaurants in radius of 250m and those with any Chinese restaurant closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as starting points for more detailed local analysis based on other factors.

Result of all these are 15 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Chinese restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to New York center but not crowded with existing restaurants (particularly Chinese) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify New York areas close to center with low number of restaurants (particularly Chinese restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Chinese restaurant. By calculating restaurant density distribution from Foursquare data we have first identified Queens borough that justify further analysis and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.